In [115]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime

In [50]:
url='https://www.marmiton.org/recettes/recette_tacos-mexicains_34389.aspx'

In [51]:
r.get(url)

<Response [200]>

In [52]:
soup=BeautifulSoup(r.get(url).content)

In [ ]:
list_of selectors=['h1','div.recipe-author__author>span','div.recipe-infos-users__rate__stars>span','ul.mrtn-tags-list>li','span.recipe-infos-users__value','div.recipe-infos,div.recipe-infos__timmings__detail','div.recipe-infos__timmings__cooking,div.recipe-infos__timmings__preparation']

In [53]:
#Recipe Name
soup.select('h1')[0].text

'Tacos mexicains'

In [54]:
#Author name
soup.select('div.recipe-author__author>span')[0].text.strip('\n ')

'noor_16706694'

In [76]:
#recipe note
soup.select('div.recipe-infos-users__rate__stars>span')[0].text
[i.text for i in soup.select('div.recipe-infos-users__rate__stars>span')]

['4.6/5']

In [56]:
#tags ('entrée, 'plat unique','plats mexicains)
#[i.text for i in soup.select('ul.mrtn-tags-list>li')]

['Entrée', 'Plat unique', 'Plats mexicains']

In [64]:
#Nb likes
soup.select('span.recipe-infos-users__value')[1].text

'15k'

In [57]:
#Nb of portions
[i.replace('\n',' ') for i in soup.select('div.recipe-infos,div.recipe-infos__timmings__detail')[0].text.split('\n\n') if len(i)>1]

['temps 55 min', ' personnes 4', ' très facile', 'bon marché']

In [71]:
#cooking time  only
[re.sub('\t|\n|\xa0','',i.text) for i in soup.select('div.recipe-infos__timmings__cooking')]


['Cuisson:25 min']

In [72]:
#Preparation time 
[re.sub('\t|\n|\xa0','',i.text) for i in soup.select('div.recipe-infos__timmings__preparation')]


['Préparation: 30 min']

In [66]:
#total time
soup.select('div.recipe-infos__timmings__total-time ')[0].text

'\nTemps Total : 55 min\t\t\t\t\t\t\t\t'

In [69]:
#nb people
soup.select('span.title-2.recipe-infos__quantity__value')[0].text

'4'

In [ ]:
#Ingredients
[i.text.strip().replace('\n','') for i in soup.select('li.recipe-ingredients__list__item')]

In [81]:
# average cost
[i.text.strip('\n') for i in soup.select('div.recipe-infos__budget')]
        

['bon marché']

In [85]:
# difficulty
[i.text.strip('\n') for i in soup.select('div.recipe-infos__level')]

['très facile']

In [ ]:
#Ustensiles
[re.sub('\n\n|\t*',' ',i.text) for i in soup.select('ul.recipe-utensils__list')]

In [ ]:
[i.text for i in soup.select('ul.recipe-utensils__list')]

In [75]:
# recipe
re.sub('\t|\n','',soup.select('ol.recipe-preparation__list')[0].text)
[i.text for i in soup.select('ol.recipe-preparation__list')]

["\n\n\t\t\tA la poêle, faire dorer l'oignon émincé dans un peu d'huile d'olive. \t\t\n\n\t\t\tRajouter la viande, assaisonner et laisser cuire 5 min. \t\t\n\n\t\t\tLaver les feuilles de laitue. \t\t\n\n\t\t\tCouper les tomates et le poivron en petits dés. \t\t\n\n\t\t\tIncorporer le tout à la poêlée avec le coulis de tomate, et poursuivre la cuisson pendant 5 min. \t\t\n\n\t\t\tEgoutter les haricots rouges et les ajouter 2 min avant la fin de cuisson. \t\t\n\n\t\t\tHors du feu, ajuster l'assaisonnement et saupoudrer généreusement de cumin; on peut aussi rajouter quelques gouttes de Tabasco. \t\t\n\n\t\t\tGarnir les tortillas de préparation et les refermer en les roulant comme des crêpes. Disposer 1 feuille de laitue sur chaque tacos avant de servir.\t\t\n"]

In [ ]:
url1='https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=50&page=4'

In [ ]:
soup1=BeautifulSoup(r.get(url1).content)

In [ ]:
[i['href'] for i in soup1.select('a.recipe-card-link')]


# Code aggregation

In [ ]:
# 14 recipes per page with page from no figure then 2 to 833
#https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=10004&page=833

In [110]:
#liste of links for the pages - start with 5pages to test before launching 833 pages...
# pb le site change de pattern de pagination 
def List_urls(NbPages):
    urls=[]
    for i in range(0,NbPages):
        url=f'https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start={4+14*i}&page={i+1}'
        urls.append(url)
        i+=1
    return urls

page_urls=List_urls(1)    
print(page_urls)

['https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=4&page=1']


In [111]:
#For one given page look for the list of recipes (links)
def find_recipe_card_link(page_urls):
    recipes_of_all_pages=[]
    for url in page_urls:
        soup=BeautifulSoup(r.get(url).content)
        # list of recipes url which are in the page
        recipes_of_all_pages.extend([f'https://www.marmiton.org'+i['href'] for i in soup.select('a.recipe-card-link')])
    return recipes_of_all_pages
        
recipes_of_all_pages=find_recipe_card_link(page_urls) 
recipes_of_all_pages

['https://www.marmiton.org/recettes/recette_original-american-cookies-de-mike_39907.aspx',
 'https://www.marmiton.org/recettes/recette_blanquette-de-veau-facile_19219.aspx',
 'https://www.marmiton.org/recettes/recette_le-crumble-aux-pommes-du-chat-qui-tousse_10534.aspx',
 'https://www.marmiton.org/recettes/recette_gateau-au-yaourt_12719.aspx',
 'https://www.marmiton.org/recettes/recette_veritable-moelleux-au-chocolat_12825.aspx',
 'https://www.marmiton.org/recettes/recette_les-timbales-de-jeanne-saumon-a-la-mousse-de-courgettes-au-micro-ondes_21864.aspx',
 'https://www.marmiton.org/recettes/recette_gateau-aux-pommes-facile_13493.aspx',
 'https://www.marmiton.org/recettes/recette_meringue-pour-les-nuls_36610.aspx',
 'https://www.marmiton.org/recettes/recette_pancakes_15299.aspx',
 'https://www.marmiton.org/recettes/recette_cookie-cup-glace-vanille-et-pepites-de-chocolat_372241.aspx',
 'https://www.marmiton.org/recettes/recette_filet-mignon-en-croute_14407.aspx',
 'https://www.marmiton.o

In [109]:
def Collect_data(recipes_of_page):
    
    
    #recipe=dict() #dictionnary , 1 row of the final dataframe (Recipe name: [list of info])  recipe={'':[]}
    list_of_recipes=[]
    recipe=[]    
    for url in recipes_of_page:
        
       
        soup=BeautifulSoup(r.get(url).content)
        
        
        # Add Recipe Name
        
        name=recipe.append([i.text for i in soup.select('h1')])
      
            
        # add author name
        recipe.append([i.text.strip('\n ') for i in soup.select('div.recipe-author__author>span')]) 
        
        #add note
        recipe.append([i.text for i in soup.select('div.recipe-infos-users__rate__stars>span')]) 
        
        #add tags
        recipe.append([i.text for i in soup.select('ul.mrtn-tags-list>li.mrtn-tag>a')])
        
        # nb likes
        try:
            recipe.append(soup.select('span.recipe-infos-users__value')[1].text)
        except IndexError:
            recipe.append()
        
        #nb portions
        recipe.append([i.text.strip('\n') for i in soup.select('span.title-2.recipe-infos__quantity__value')])
        
        # difficulty
        recipe.append([i.text.strip('\n') for i in soup.select('div.recipe-infos__level')])
        
        #add total cooking time
        recipe.append([i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__total-time ')])
        
        # preparation time
        recipe.append([i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__preparation')])
            
        # cooking only time
        recipe.append([i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__cooking')])

        # average cost
        recipe.append([i.text.strip('\n') for i in soup.select('div.recipe-infos__budget')])
        
        # add url
        recipe.append(url)
        
        #ingredients
        recipe.append([i.text.strip('\n') for i in soup.select('li.recipe-ingredients__list__item')])
        
        #recipe (text)
        recipe.append([i.text.strip('\n') for i in soup.select('ol.recipe-preparation__list')])
        
        #check
        print(recipe)
        
        list_of_recipes.append(recipe)
        recipe=[]
    return list_of_recipes

#Collect_data(recipes_of_all_pages)

In [112]:
# new method
        
def Scrap_data(url):   
    
        
    soup=BeautifulSoup(r.get(url).content)
    
    # Recipe Name
    name=[i.text for i in soup.select('h1')][0]
       
    # Author name
    author_name=[i.text.strip('\n ') for i in soup.select('div.recipe-author__author>span')][0]
        
    #Rating
    rating= [i.text for i in soup.select('div.recipe-infos-users__rate__stars>span')][0]
        
    #Tags
    tags=[i.text for i in soup.select('ul.mrtn-tags-list>li.mrtn-tag>a')]
        
    # nb likes
    try:
        nblikes=soup.select('span.recipe-infos-users__value')[1].text
    except IndexError:
        nblikes=''
        
    #nb portions
    nbportions=[i.text.strip('\n') for i in soup.select('span.title-2.recipe-infos__quantity__value')][0]
        
    # difficulty
    difficulty=[i.text.strip('\n') for i in soup.select('div.recipe-infos__level')][0]
        
    #Total cooking time
    totaltime=[i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__total-time ')][0]
        
    # preparation time
    preparationtime=[i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__preparation')][0]
            
    # cooking only time
    cookingtime=[i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__cooking')][0]

    # cost range
    costrange=[i.text.strip('\n') for i in soup.select('div.recipe-infos__budget')][0]
        
    # add url
    link=url
        
    #list of ingredients
    loi=[i.text.strip('\n') for i in soup.select('li.recipe-ingredients__list__item')]
        
    #recipe (text)
    recipedescr=[i.text.strip('\n') for i in soup.select('ol.recipe-preparation__list')]
        
    dct={'Recipe_name':name,\
         'Author_name':author_name,\
         'Rating':rating,\
         'Tags':tags,\
         'Nb_likes':nblikes,\
         'Nb_portions':nbportions,\
         'Difficulty':difficulty,\
         'Total_cooking_time':totaltime,\
         'Preparation_time':preparationtime,\
         'Cooking_time':cookingtime,\
         'Cost_range':costrange,\
         'List_of_ingredients':loi,\
         'Recipe_description':recipedescr,\
         'Link':link}
    return dct


Scrap_data('https://www.marmiton.org/recettes/recette_original-american-cookies-de-mike_39907.aspx')


{'Recipe_name': 'Original American Cookies de Mike',
 'Author_name': 'nathan_15813882',
 'Rating': '4.7/5',
 'Tags': ['Végétarien',
  'Rapide',
  'Petits gâteaux',
  'Gâteau',
  'Cookie',
  'Recette américaine'],
 'Nb_likes': '121k',
 'Nb_portions': '15',
 'Difficulty': 'très facile',
 'Total_cooking_time': 'Temps Total : 15 min\t\t\t\t\t\t\t\t',
 'Preparation_time': 'Préparation\xa0: 5 min\t\t\t\t\t\t\t\t\t\t',
 'Cooking_time': 'Cuisson\xa0:\n\n\t\t\t\t\t\t\t\t\t\t\t\t10 min\t\t\t\t\t\t\t\t\t\t\t',
 'Cost_range': 'bon marché',
 'List_of_ingredients': ['250\n\n\n\ng de farine  ',
  '90\n\n\n\ng de sucre de canne roux (ou cassonade) ',
  '1\n\n\n\nsachet de sucre vanillé  ',
  '1\n\n\n\npincée de sel  ',
  '1/2\n\n\n\nsachet de levure  ',
  '1\n\n\n\n oeuf  ',
  '125\n\n\n\ng de beurre doux  ',
  '2\n\n\n\ncuillères à café de miel  ',
  'Chocolat (à volonté) '],
 'Recipe_description': ["\t\t\tMélangez la farine, les sucres, le sel et la levure dans un grand saladier.\t\t\n\n\t\t\tFaites

In [113]:
# Get all recipes for all pages
data=[]

for ix, i in enumerate(recipes_of_all_pages):
    output=Scrap_data(i)
    data.append(output)
    if ix%5==0:
        time.sleep(2)

NameError: name 'time' is not defined

In [ ]:
df=pd.DataFrame(data)

In [47]:
url='https://www.marmiton.org/recettes/recette_original-american-cookies-de-mike_39907.aspx'

In [48]:
soup=BeautifulSoup(r.get(url).content) 


In [30]:
a=soup.select('div.recipe-author__author>span')[0].text.strip('\n ')
a
toto=[]
toto.append(a)
print(toto)

['nathan_15813882']


In [22]:
#soup.select('h1','div.recipe-author__author>span','div.recipe-infos-users__rate__stars>span','ul.mrtn-tags-list>li','div.recipe-infos,div.recipe-infos__timmings__detail','span.recipe-infos-users__value','div.recipe-infos__timmings__cooking,div.recipe-infos__timmings__preparation','li.recipe-ingredients__list__item','ol.recipe-preparation__list')

In [ ]:
def create_dataFrame(recipe):
  pd.DataFrame(recipe)

In [ ]:
#'d4 = {}' 'for d in (d1, d2, d3): d4.update(d)'

In [ ]:
# Error Handling function

In [ ]:
# Launch function terminal

In [ ]:
url_page='https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=18&page=2'
soup=BeautifulSoup(r.get(url_page).content)
[f'https://www.marmiton.org'+i['href'] for i in soup.select('a.recipe-card-link')]

In [ ]:
https://www.marmiton.org/recettes/recette_chouquettes_30622.aspx

In [ ]:
#